In [1]:
# current work directory

%pwd

'f:\\Project\\Text_Summarization\\research'

In [3]:
import os
os.chdir('../')
%pwd

'f:\\Project\\Text_Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    dataset_path: Path
    tokenizer_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directories, read_yaml

class ConfigManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) ->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            dataset_path= os.path.join(self.config.data_ingestion.root_dir, 
                         self.config.dataset_name+'_dataset'),
                         tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config

In [6]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset ,load_from_disk

In [14]:
class DataTranformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name, use_fast = False)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation= True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length =128, truncation= True)
        return{
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [15]:
try:
    config_manager = ConfigManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTranformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-08 19:06:39,350 : INFO : common : yaml file : config\config.yml loaded succefully]
[2024-01-08 19:06:39,353 : INFO : common : yaml file : params.yml loaded succefully]
[2024-01-08 19:06:39,355 : INFO : common : Created Director at: artifacts]
[2024-01-08 19:06:39,357 : INFO : common : Created Director at: artifacts/data_transformation]


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.